##Implementation of GRU on the time-series dataset

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Load test features and labels
X_test = np.load('/content/drive/MyDrive/Dataset of ai/rolling_window_sequnces_1.npy')  # Replace with actual file path
metadata_test = pd.read_csv("/content/drive/MyDrive/Dataset of ai/sequence_metadata_with_RUL (1).csv")  # Replace with actual file path
y_test = metadata_test["RUL"].values
print("Test feature shape:", X_test.shape)
print("Test target shape:", y_test.shape)

Test feature shape: (17631, 30, 66)
Test target shape: (17631,)


In [ ]:
# Split the data into train and validation sets (assuming no separate train set given)
X_train, X_val, y_train, y_val = train_test_split(X_test, y_test, test_size=0.3, random_state=42)

In [ ]:

# Define GRU model creation function
def create_gru_model(input_shape, units=64, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(GRU(units, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

In [ ]:
# Create the model
model = create_gru_model(input_shape=(X_train.shape[1], X_train.shape[2]), units=64, learning_rate=0.001, dropout_rate=0.2)

# Callbacks for early stopping
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 8016.9370 - mae: 78.4196 - val_loss: 6364.6011 - val_mae: 68.1893
Epoch 2/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - loss: 6005.4126 - mae: 65.5869 - val_loss: 5068.2461 - val_mae: 59.2524
Epoch 3/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 4760.8330 - mae: 57.0085 - val_loss: 4050.9717 - val_mae: 51.8954
Epoch 4/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 3870.2195 - mae: 50.6451 - val_loss: 3244.0984 - val_mae: 45.6268
Epoch 5/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - loss: 3100.3445 - mae: 44.6026 - val_loss: 2596.6348 - val_mae: 40.3759
Epoch 6/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - loss: 2464.6675 - mae: 39.3324 - val_loss: 2023.4224 - val_mae: 35.3280
Epoch 7/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 1938.1782 - mae: 34.6324 - val_loss: 1572.5248 - val_mae: 30.8407
Epoch 8/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 1497.7661 - mae: 30.2013 - val_loss: 1145.4210 - v

In [ ]:

# Evaluate on validation set
val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation MSE: {val_loss:.4f}, MAE: {val_mae:.4f}')

# Predict on test set
y_pred = model.predict(X_test).flatten()

# Calculate test MSE
test_mse = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {test_mse:.4f}')

Validation MSE: 120.8157, MAE: 8.1736
551/551 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
Test MSE: 106.5865


In [ ]:
model.save("/content/drive/MyDrive/Dataset of ai/best_gru_model.keras")